In [1]:
from nn import QLearningAgent, QNetwork
from verify_data import FishGame, ParseError

In [2]:
import os

memories = []
for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    with open(filepath, 'r') as f:
        try:
            print(f"{filename}")
            game = FishGame(f.readlines())
            for player in game.players:
                memories += game.memory(player)
        except ParseError as e:
            print(f"{filename}: {e}")
            break

12-10_11:07.txt
12-3_14:05.txt
12-3_11:30.txt
12-3_14:27.txt
12-6_11:08.txt
12-3_11:12.txt
1-15_11:15.txt


In [3]:
agent = QLearningAgent(memories)
agent.train()

/Users/Yourui/Documents/Fish/nn.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  return torch.FloatTensor(x).to(self.device)
/Users/Yourui/Documents/Fish/nn.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'call_set': F.softmax(call_set.masked_fill(~action_masks['call_set'], -1e9)),


tensor([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [False,  True,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [False,  True, False,  True],
        [ True,  True,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [False,  True, False, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True, False],
        [ True,  True, False,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True, False,  True],
        [ True,  True,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [False,  True,  True, False],
        [ Tr

/Users/Yourui/Documents/Fish/nn.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'ask_person': F.softmax(ask_person.masked_fill(~action_masks['ask_person'], -1e9)),
/Users/Yourui/Documents/Fish/nn.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'ask_set': F.softmax(ask_set.masked_fill(~action_masks['ask_set'], -1e9)),
/Users/Yourui/Documents/Fish/nn.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'ask_card': F.softmax(ask_card),
/Users/Yourui/Documents/Fish/nn.py:80: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = lambda prev, next, player, reward: F.mse_loss(sum(prev * player),


tensor([0.2557, 0.2700, 0.2212, 0.2532], device='mps:0',
       grad_fn=<SelectBackward0>) tensor([0.2564, 0.2722, 0.2221, 0.2493], device='mps:0',
       grad_fn=<SelectBackward0>) None


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn